In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# set up arrays
dx = 1000
x = np.arange(0, 100000, dx)
x_node = x + (dx/2)

Hstar = 0.5
etab = np.zeros(len(x)) # bedrock elevation, m
etab[:] = np.linspace(10, 0.1, len(x)) # need to start with slight bedrock slope
H = Hstar + np.zeros(len(x))
eta = etab + H

In [3]:
# set up parameters
# K_ig = np.zeros(len(x))
# K_ig[:24] = 1e-7
# K_sed = np.zeros(len(x))
# K_sed[24:] = 1e-6
K = np.zeros(len(x))
K[:] = 1e-6

# beta_ig = np.zeros(len(x))
# beta_ig[:24] = 4e-7
# beta_sed = np.zeros(len(x))
# beta_sed[24:] = 4e-6
beta = np.zeros(len(x))
beta[:] = 1e-4
# beta[:] = 1e-6

# psi = np.zeros(len(x))
# psi[:14] = 4e-7
# psi[14:] = 4e-6
psi = 1e-4
# psi = 1e-6
# psi = 1e-5
gamma = 0.3

r = 5. # runoff rate
D = 0.015 # grain size
phi = 0.45 # porosity
baselevel_rate = 0.0055 # baselevel fall rate

In [4]:
kqs = 0.041
I = 0.01
kxb = 25
Pxb = 1./5.
kb = 8.3e-8

B_valley = kxb * (x_node**Pxb)
B_basin = (2./3.) * x_node

area = (1./3.) * x**2
# Q = r * x * B
Q = r * area

In [5]:
def model(x, dx, x_node,
          K, beta,
          D, phi, Hstar, psi, gamma,
          kb, kqs, I, r, B_valley, B_basin, Q,
          etab, H, eta,
          baselevel_rate,
          num_steps = 1):
    
    q = Q/B_valley
    # dt = 0.2 * dx * dx / (kqs * q[-1]) # try changing to 10, 5, or 1 if this is unstable
    dt = 5
    
    run_duration = dt * num_steps
    cum_time = 0.0
    
    Qs = np.zeros(len(x))
    qs = np.zeros(len(x))
    Eb_p = np.zeros(len(x))
    Eb_a = np.zeros(len(x))
    Eb = np.zeros(len(x))
    Eh = np.zeros(len(x))
    E = np.zeros(len(x))
    ql = np.zeros(len(x))
    
    H[-1] = 0
    E[-1] = baselevel_rate
    q[0] = 0
    Qs[0] = 0
    ql[0] = 0
    
    while cum_time < run_duration:
        
        # drop baselevel
        eta[-1] -= baselevel_rate * dt
        
        # set boundary conditions
        etab[-1] = eta[-1]
        
        # calc slope
        S = -np.diff(eta)/dx
        
        # calc bed exposure
        alpha = np.exp(-H/Hstar)
        
        # calc Rh
#         width = (kb * Q[1:] * (S ** (7/6))) / (D**(3/2))
#         depth = (0.09801 * D) / S
#         Rh_efficiency = 1 - np.exp(-width/(2*depth))
        
        # calc sed transport
        Qs[1:] = kqs * I * Q[1:] * S**(7./6.) * (1 - alpha[:-1]) # * Rh_efficiency
        qs[1:] = Qs[1:] / B_valley[1:]
        
        # calc bedrock erosion
        Eb_p[:-1] = K[1:] * Q[1:] * S * alpha[:-1]
        Eb_a[:-1] = beta[1:] * Qs[1:] * alpha[:-1]
        Eb[:-1] = (Eb_p[:-1] + Eb_a[:-1]) / B_valley[:-1]
        
        # calc grain attrition
        atr = Qs * psi
        
        # calc equilibrium lateral supply
        ql[1:] = ((baselevel_rate * gamma) / psi) * (1 - np.exp(-psi/(3 * x[1:])))
        # ql[24:] = 0
        
        # calc sediment erosion rate
        Eh[:-1] = 1/((1 - phi) * B_valley[:-1]) * (np.diff(Qs)/dx + atr[1:] - Eb_p[:-1] - ql[1:])
        
        # calc total erosion rate
        E[:-1] = Eb[:-1] + Eh[:-1]
        
        # update topography
        etab[:-1] -= Eb[:-1] * dt
        H[:-1] -= Eh[:-1] * dt
        H[H<0] = 0
        eta[:-1] = etab[:-1] + H[:-1]
        
        # update elapsed time
        cum_time += dt
        
    print(cum_time)
        
    return (dt, S, q, alpha, Qs, qs, Eb_p, Eb_a, Eb, atr, ql, Eh, E, etab, H, eta)

In [6]:
#now write a test
num_steps = 1000000
test_run = [x, dx, x_node, 
            K, beta, 
            D, phi, Hstar, psi, gamma,
            kb, kqs, I, r, B_valley, B_basin, Q,
            etab, H, eta,
            baselevel_rate,
            num_steps]

# and run it
(dt, S, q, alpha, Qs, qs, Eb_p, Eb_a, Eb, atr, ql, Eh, E, etab, H, eta) = model(*test_run)

500000.0


In [7]:
Eb

array([0.00062319, 0.00186859, 0.00270449, 0.00327644, 0.00369391,
       0.00401154, 0.00426006, 0.00445844, 0.00461921, 0.00475105,
       0.00486019, 0.00495125, 0.00502774, 0.00509234, 0.00514717,
       0.0051939 , 0.00523387, 0.00526818, 0.00529772, 0.00532321,
       0.00534526, 0.00536439, 0.005381  , 0.00539545, 0.00540806,
       0.00541906, 0.00542868, 0.00543711, 0.00544449, 0.00545097,
       0.00545666, 0.00546167, 0.00546608, 0.00546996, 0.00547338,
       0.0054764 , 0.00547907, 0.00548143, 0.00548351, 0.00548535,
       0.00548699, 0.00548843, 0.00548971, 0.00549085, 0.00549186,
       0.00549275, 0.00549355, 0.00549425, 0.00549488, 0.00549544,
       0.00549593, 0.00549638, 0.00549677, 0.00549712, 0.00549743,
       0.00549771, 0.00549796, 0.00549818, 0.00549837, 0.00549855,
       0.0054987 , 0.00549884, 0.00549897, 0.00549908, 0.00549918,
       0.00549927, 0.00549935, 0.00549942, 0.00549948, 0.00549954,
       0.00549959, 0.00549963, 0.00549967, 0.00549971, 0.00549

In [8]:
S

array([0.13043898, 0.14237457, 0.13249042, 0.12143385, 0.11140769,
       0.10256378, 0.0947687 , 0.08786465, 0.08171408, 0.07620407,
       0.07124284, 0.06675553, 0.06268055, 0.05896676, 0.05557126,
       0.05245778, 0.04959537, 0.04695746, 0.04452106, 0.04226618,
       0.04017533, 0.03823314, 0.03642605, 0.03474201, 0.03317034,
       0.03170147, 0.03032685, 0.02903879, 0.02783037, 0.02669534,
       0.02562805, 0.02462337, 0.02367664, 0.02278362, 0.02194044,
       0.02114357, 0.02038978, 0.01967611, 0.01899982, 0.01835844,
       0.01774965, 0.01717134, 0.01662156, 0.01609851, 0.0156005 ,
       0.01512601, 0.0146736 , 0.01424195, 0.01382982, 0.01343608,
       0.01305965, 0.01269956, 0.01235487, 0.01202474, 0.01170835,
       0.01140496, 0.01111387, 0.01083443, 0.01056601, 0.01030806,
       0.01006004, 0.00982143, 0.00959176, 0.0093706 , 0.00915752,
       0.00895213, 0.00875407, 0.00856298, 0.00837853, 0.00820043,
       0.00802837, 0.00786208, 0.00770131, 0.0075458 , 0.00739

In [9]:
# now calc width
width = (kb * Q[1:] * (S**(7./6.))) / (D**(3./2.))
width

array([   6.9946383 ,   30.98758694,   64.10831447,  102.95315763,
        145.47811423,  190.21824242,  236.09952476,  282.32745656,
        328.31295365,  373.62086954,  417.9330562 ,  461.02117991,
        502.72631066,  542.94335233,  581.60901594,  618.69243827,
        654.18780798,  688.10853657,  720.48263123,  751.34901206,
        780.75457683,  808.75186147,  835.39717776,  860.74913486,
        884.86747052,  907.81213293,  929.64256554,  950.41715639,
        970.19282099,  989.02469319, 1006.96590352, 1024.06742795,
       1040.3779934 , 1055.94402844, 1070.80964998, 1085.01667824,
       1098.60467378, 1111.6109914 , 1124.07084667, 1136.01739177,
       1147.48179768, 1158.49334066, 1169.07949095, 1179.26600248,
       1189.07700224, 1198.53507844, 1207.66136687, 1216.47563473,
       1224.99636172, 1233.24081789, 1241.22513838, 1248.96439452,
       1256.47266161, 1263.76308319, 1270.84793175, 1277.73866625,
       1284.44598611, 1290.97988223, 1297.34968474, 1303.56410

In [10]:
# now calc depth
depth = (0.09801 * D) / S
depth

array([0.01127079, 0.01032593, 0.01109627, 0.01210659, 0.01319613,
       0.01433401, 0.01551303, 0.01673199, 0.01799139, 0.01929228,
       0.02063576, 0.0220229 , 0.02345464, 0.02493184, 0.02645522,
       0.0280254 , 0.02964289, 0.03130813, 0.03302145, 0.03478313,
       0.03659335, 0.03845224, 0.04035986, 0.0423162 , 0.04432122,
       0.04637482, 0.04847684, 0.05062711, 0.05282539, 0.05507141,
       0.05736489, 0.05970549, 0.06209286, 0.06452663, 0.0670064 ,
       0.06953177, 0.07210229, 0.07471753, 0.07737703, 0.08008034,
       0.08282698, 0.08561648, 0.08844836, 0.09132214, 0.09423735,
       0.09719351, 0.10019013, 0.10322674, 0.10630289, 0.10941809,
       0.1125719 , 0.11576386, 0.11899353, 0.12226047, 0.12556425,
       0.12890445, 0.13228066, 0.13569247, 0.1391395 , 0.14262135,
       0.14613766, 0.14968805, 0.15327216, 0.15688966, 0.16054019,
       0.16422344, 0.16793907, 0.17168678, 0.17546627, 0.17927723,
       0.18311938, 0.18699245, 0.19089616, 0.19483024, 0.19879